# Databases - week11
install package: mysql-connector-python

In [1]:
! pip install mysql-connector-python

## Connectie maken met een databank

In [2]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()

print(dbconnection.database)
cursor.close()
dbconnection.close()

movies_db


## CREATE OPERATION

In [3]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()
query = "INSERT INTO actors(name) VALUES ('Kevin Hart')"
cursor.execute(query)
dbconnection.commit()

cursor.close()
dbconnection.close()

In [5]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()

query = "INSERT INTO actors(name) VALUES(%s)"
data = [["Leonardo Dicaprio"], ["Samuel L Jackson"], ["Margot Robbie"], ["Angelina Jolie"], ["Jennifer Aniston"]]

cursor.executemany(query, data)


#ook mogelijks als je een categorie hebt in je db met meerdere waarden

query = "INSERT INTO movies(title, synopsis) VALUES (%s, %s)"

data = ("The Matrix", "John wick time traveller")
cursor.execute(query, data)

#met meerdere inputs voor meerdere categoriën

movies = [("Inception", "Daydream Cilian Murphy, blauwe ogen"),
          ("Interstellar", "louche planeten verkennen en dochter zien sterven, defying laws of physics"),
          ("Jurassic Park", "Exerimental dinosaur zoo"),
          ("Star Wars", "boerenjongen gaat bij de rebellen")]

#iedere film toevoegen
for movie in movies:
    cursor.execute(query, movie)

dbconnection.commit()
cursor.close()
dbconnection.close()

## READ OPERATION

In [14]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()
query = "SELECT * FROM actors"
cursor.execute(query)

#cursor.execute("SELECT * FROM actors")

data = cursor.fetchall()
print(data)

for actor in data:
    print(actor) #print alles

for enkelacteur in data:
    print(enkelacteur[1]) #0 --> id

for id, name in data:
    print(id)
    print(name)

cursor.close()
dbconnection.close()

[(1, 'Michelle Yeoh'), (2, 'Stephan James'), (3, 'Jamie Lee Curtis'), (4, 'Tom Cruise'), (5, 'Kevin Hart'), (6, 'Leonardo Dicaprio'), (7, 'Samuel L Jackson'), (8, 'Margot Robbie'), (9, 'Angelina Jolie'), (10, 'Jennifer Aniston')]
(1, 'Michelle Yeoh')
(2, 'Stephan James')
(3, 'Jamie Lee Curtis')
(4, 'Tom Cruise')
(5, 'Kevin Hart')
(6, 'Leonardo Dicaprio')
(7, 'Samuel L Jackson')
(8, 'Margot Robbie')
(9, 'Angelina Jolie')
(10, 'Jennifer Aniston')
Michelle Yeoh
Stephan James
Jamie Lee Curtis
Tom Cruise
Kevin Hart
Leonardo Dicaprio
Samuel L Jackson
Margot Robbie
Angelina Jolie
Jennifer Aniston
1
Michelle Yeoh
2
Stephan James
3
Jamie Lee Curtis
4
Tom Cruise
5
Kevin Hart
6
Leonardo Dicaprio
7
Samuel L Jackson
8
Margot Robbie
9
Angelina Jolie
10
Jennifer Aniston


In [17]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()

query = """select movies.title, actors.name from movies
                inner join movie_actor on movies.id = movie_actor.movie_id
                inner join actors on actors.id = movie_actor.actor_id"""

cursor.execute(query)
data = cursor.fetchall()

for row in data:
    print(row)

for movie, actor in data:
    print(actor, "speelt in", movie)

cursor.close()
dbconnection.close()

('Everything Everywhere All at Once', 'Michelle Yeoh')
('Everything Everywhere All at Once', 'Stephan James')
('Everything Everywhere All at Once', 'Jamie Lee Curtis')
('Top Gun: Maverick', 'Tom Cruise')
Michelle Yeoh speelt in Everything Everywhere All at Once
Stephan James speelt in Everything Everywhere All at Once
Jamie Lee Curtis speelt in Everything Everywhere All at Once
Tom Cruise speelt in Top Gun: Maverick


### check slides voor querying in SQL

## UPDATE OPERATION
Jumanji --> eerst toevoegen, daarna updaten
Board game brings jungle chaos, adventure and danger

In [19]:
import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()

#cursor.execute("INSERT INTO movies(title) VALUES('Jumanji')")
query = "UPDATE movies SET synopsis = 'Board game brings jungle chaos, adventure and danger' WHERE title like 'Jumanji'"
cursor.execute(query)
dbconnection.commit()
cursor.close()
dbconnection.close()


## SCRAPE DATA NAAR DATABASE
https://en.wikipedia.org/wiki/List_of_Academy_Award%E2%80%93winning_films

In [22]:
import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films"
response = requests.get(url)
soup = BeautifulSoup(response.text)

import mysql.connector

dbconnection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database = "movies_db"
)

cursor = dbconnection.cursor()
tabel = soup.find("table")
rijen = tabel.find_all("tr")

for rij in rijen:
    td = rij.find_all("td")
    if len(td) > 0:
        title = td[0].get_text().strip().replace("'", "")
        year = td[1].get_text().strip()
        awards = td[2].get_text().strip()
        nom = td[3].get_text().strip()

        #check voor dubbels
        query = "SELECT * FROM movies where title like'" + title + "'"
        cursor.execute(query)
        data = cursor.fetchall()

        if len(data) == 0:
            query = "INSERT into movies(title) VALUES('" + title + "')"
            cursor.execute(query)
            dbconnection.commit()

            movie_id = cursor.lastrowid
            query = "INSERT INTO awards(movie_id, awardshow_id, year, awards, nominations) VALUES (%s, %s, %s, %s, %s)"

            values = (movie_id, 1, year, awards, nom)
            cursor.execute(query, values)
            dbconnection.commit()

cursor.close()
dbconnection.close()